Notebook putting together concepts from all modeling notebook to construct final model

In [1]:
import pandas as pd
import numpy as np
import seaborn as sb
import matplotlib.pyplot as plt
from pandas.plotting import scatter_matrix
import matplotlib.cm as cm
%matplotlib inline

/Users/pauldefusco/anaconda2/envs/py27/lib/python2.7/site-packages/statsmodels/compat/pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools


In [169]:
from sklearn import linear_model
from sklearn.metrics import r2_score, mean_squared_error,mean_absolute_error
from sklearn.model_selection import cross_val_score, cross_validate, train_test_split, GridSearchCV
from sklearn.preprocessing import LabelEncoder, LabelBinarizer, PolynomialFeatures, MinMaxScaler, StandardScaler
from sklearn.pipeline import Pipeline
from sklearn import preprocessing
from sklearn.neighbors import LocalOutlierFactor
from sklearn.feature_selection import RFE, f_regression, RFECV
from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor

In [95]:
listings = pd.read_csv('Datasources/listings_augmented/listings_augmented_2018-05-23_V5.csv',low_memory=False)
listings = listings.drop(listings.index[4216:4219])
listings.index = [i for i in range(len(listings))]

In [96]:
#Excluded variables from the featuresExploration notebook
%store -r excluded

In [97]:
cols = [i for i in listings.columns if i not in excluded]
X = listings[cols]

In [98]:
def scale_data(X):
    return pd.DataFrame(preprocessing.scale(X),columns = X.columns)

In [99]:
def normalize_data(X):
    return pd.DataFrame(preprocessing.normalize(X),columns = X.columns)

In [101]:
X[X.columns[X.isnull().any()]].isnull().sum()

bathrooms               8
bedrooms                3
beds                    3
reviews_per_month    1563
dtype: int64

In [102]:
X = X.fillna(X.mean())

In [103]:
X_scaled = scale_data(X)
X_normed = normalize_data(X)

/Users/pauldefusco/anaconda2/envs/py27/lib/python2.7/site-packages/sklearn/preprocessing/data.py:181: UserWarning: Numerical issues were encountered when scaling the data and might not be solved. The standard deviation of the data is probably very close to 0. 
  warnings.warn("Numerical issues were encountered "


In [105]:
y_scaled = X_scaled['price_y'].fillna(X_scaled['price_y'].mean())
X_scaled = X_scaled[X_scaled.columns.drop(X_scaled[list(X_scaled.filter(regex='price'))])]

In [137]:
y_normed = X_normed['price_y'].fillna(X_normed['price_y'].mean())
X_normed = X_normed[X_normed.columns.drop(X_normed[list(X_normed.filter(regex='price'))])]

Step1: Simple Model with Regression -- No Validation Set

In [157]:
def eval_metrics(scores):
    print 'Training R2 Mean: ',scores['train_r2'].mean()
    print 'Validation R2 Mean: ',scores['test_r2'].mean()
    print 'Validation R2 STdev: ',scores['test_r2'].std()
    print '--'
    print 'Training RMSE Mean: ', np.sqrt(-scores['train_neg_mean_squared_error'].mean())
    print 'Validation RMSE Mean: ', np.sqrt(-scores['test_neg_mean_squared_error'].mean())
    print 'Validation RMSE STdev: ',scores['test_neg_mean_squared_error'].std()
    print '--'
    print 'Training MAE Mean: ', -scores['train_neg_mean_absolute_error'].mean()
    print 'Validation MAE Mean: ', -scores['test_neg_mean_absolute_error'].mean()
    print 'Validation MAE STdev: ',scores['test_neg_mean_absolute_error'].std()

In [138]:
X_train, X_test, y_train, y_test = train_test_split(X_normed,y_normed, test_size=0.3, random_state=42)

In [139]:
lin_reg = linear_model.LinearRegression(fit_intercept=True, normalize=False)

In [140]:
lin_reg.fit(X_train, y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=False)

In [141]:
predictions_train = model.predict(X_train)

In [142]:
print 'Training R2: ',r2_score(y_train, predictions_train)
print 'Training RMSE: ',np.sqrt(mean_squared_error(y_train, predictions_train))
print 'Training MAE: ',mean_absolute_error(y_train, predictions_train)

Training R2:  -2.3551666642089892
Training RMSE:  0.08373902957449732
Training MAE:  0.06927594512135013


In [143]:
tree_reg = DecisionTreeRegressor()

In [144]:
tree_reg.fit(X_train,y_train)

DecisionTreeRegressor(criterion='mse', max_depth=None, max_features=None,
           max_leaf_nodes=None, min_impurity_decrease=0.0,
           min_impurity_split=None, min_samples_leaf=1,
           min_samples_split=2, min_weight_fraction_leaf=0.0,
           presort=False, random_state=None, splitter='best')

In [145]:
predictions_train = tree_reg.predict(X_train)

In [146]:
print 'Training R2: ',r2_score(y_train, predictions_train)
print 'Training RMSE: ',np.sqrt(mean_squared_error(y_train, predictions_train))
print 'Training MAE: ',mean_absolute_error(y_train, predictions_train)

Training R2:  0.999994055114309
Training RMSE:  0.00011146605446533172
Training MAE:  5.394726221555853e-05


Using Cross Validation

In [158]:
scores_tree = cross_validate(tree_reg, X_train, y_train, cv=10, 
                         scoring=('r2', 'neg_mean_squared_error','neg_mean_absolute_error'))

In [159]:
scores_lin = cross_validate(lin_reg, X_train, y_train, cv=10, 
                         scoring=('r2', 'neg_mean_squared_error','neg_mean_absolute_error'))

In [162]:
print 'Evaluation Metrics for Tree Regression with CV: '
eval_metrics(scores_tree)
print '----'
print '----'
print 'Evaluation Metrics for Linear Regression with CV: '
eval_metrics(scores_lin)

Evaluation Metrics for Tree Regression with CV: 
Training R2 Mean:  0.9999944461386185
Validation R2 Mean:  0.657641608832561
Validation R2 STdev:  0.08791131904557492
--
Training RMSE Mean:  0.0001077300454641731
Validation RMSE Mean:  0.026441820684449675
Validation RMSE STdev:  0.00012596401570330346
--
Training MAE Mean:  5.150908759759514e-05
Validation MAE Mean:  0.01772835363428391
Validation MAE STdev:  0.0012651006817141507
----
----
Evaluation Metrics for Linear Regression with CV: 
Training R2 Mean:  0.8075767638994595
Validation R2 Mean:  0.7709047894392287
Validation R2 STdev:  0.04989920051367722
--
Training RMSE Mean:  0.020051486174855943
Validation RMSE Mean:  0.02165667738433052
Validation RMSE STdev:  6.913399806464409e-05
--
Training MAE Mean:  0.014874170976446836
Validation MAE Mean:  0.015694245094573334
Validation MAE STdev:  0.0008972703015145503


Now trying out RandomForest Regression

In [163]:
for_reg = RandomForestRegressor()

In [165]:
for_reg.fit(X_train,y_train)

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
           oob_score=False, random_state=None, verbose=0, warm_start=False)

In [166]:
scores_lin = cross_validate(for_reg, X_train, y_train, cv=10, 
                         scoring=('r2', 'neg_mean_squared_error','neg_mean_absolute_error'))

In [167]:
print 'Evaluation Metrics for Random Forest Regression with CV: '
eval_metrics(scores_lin)

Evaluation Metrics for Random Forest Regression with CV: 
Training R2 Mean:  0.9682268798544322
Validation R2 Mean:  0.828749722414792
Validation R2 STdev:  0.024450887030082923
--
Training RMSE Mean:  0.008147296466217185
Validation RMSE Mean:  0.018795693424879637
Validation RMSE STdev:  3.5346718415424504e-05
--
Training MAE Mean:  0.005343429924809734
Validation MAE Mean:  0.01320388435711455
Validation MAE STdev:  0.0005576794587710496


The Random Forest Regressor Does very well on both Training and Validation - a promising model!

In [195]:
param_grid = [
    {'n_estimators':[3,10,30], 'max_features':[2,4,6,8,10,12]},
    {'bootstrap': [True,False], 'n_estimators':[3,10], 'max_features':[2,4,6,8,10,12]},
]

In [196]:
new_for_reg = RandomForestRegressor()

In [209]:
gs = GridSearchCV(new_for_reg, param_grid, cv=4, scoring='neg_mean_absolute_error')

In [210]:
gs.fit(X_train, y_train)

GridSearchCV(cv=4, error_score='raise',
       estimator=RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
           oob_score=False, random_state=None, verbose=0, warm_start=False),
       fit_params=None, iid=True, n_jobs=1,
       param_grid=[{'n_estimators': [3, 10, 30], 'max_features': [2, 4, 6, 8]}, {'n_estimators': [3, 10], 'max_features': [2, 4, 6, 8], 'bootstrap': [True, False]}],
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring='neg_mean_absolute_error', verbose=0)

In [211]:
gs.best_params_

{'bootstrap': False, 'max_features': 8, 'n_estimators': 10}

In [212]:
gs.best_estimator_

RandomForestRegressor(bootstrap=False, criterion='mse', max_depth=None,
           max_features=8, max_leaf_nodes=None, min_impurity_decrease=0.0,
           min_impurity_split=None, min_samples_leaf=1,
           min_samples_split=2, min_weight_fraction_leaf=0.0,
           n_estimators=10, n_jobs=1, oob_score=False, random_state=None,
           verbose=0, warm_start=False)

In [214]:
gs_results = gs.cv_results_

In [215]:
gs_results.keys()

['std_train_score',
 'rank_test_score',
 'split1_train_score',
 'param_bootstrap',
 'split2_train_score',
 'split3_test_score',
 'std_score_time',
 'split2_test_score',
 'mean_score_time',
 'mean_fit_time',
 'split3_train_score',
 'split0_train_score',
 'std_test_score',
 'mean_train_score',
 'split0_test_score',
 'mean_test_score',
 'param_max_features',
 'params',
 'std_fit_time',
 'param_n_estimators',
 'split1_test_score']

In [217]:
print 'Avg Mean Train Score: ', -gs_results['mean_train_score'].mean()
print 'Avg Mean Test Score: ', -gs_results['mean_test_score'].mean()

Avg Mean Train Score:  0.005463454236924642
Avg Mean Test Score:  0.01835664386283806


In [222]:
imp = gs.best_estimator_.feature_importances_

In [231]:
from operator import itemgetter
pd.DataFrame(sorted(zip(X.columns, imp),reverse=True,key=itemgetter(1)), columns=['Feature', 'Importance']).head()

,Feature,Importance
0,accommodates,0.100005
1,bedrooms,0.075525
2,bathrooms,0.068511
3,events_within_5_km,0.063938
4,events_within_10_km,0.055819


Now testing with Test Dataset:

In [233]:
best_model = gs.best_estimator_

In [235]:
test_predictions = best_model.predict(X_test)

In [236]:
print 'Test R2: ',r2_score(y_test, test_predictions)
print 'Test RMSE: ',np.sqrt(mean_squared_error(y_test, test_predictions))
print 'Test MAE: ',mean_absolute_error(y_test, test_predictions)

Training R2:  0.8081733822858045
Training RMSE:  0.020354095971646037
Training MAE:  0.014181521331913057
